## Analysis of 2CEE_01 10k timesteps from 5K to 300K

In [16]:
# Import all necessary modules/libraries
from statistics import mean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Create class to read and format csv
class LammpsThermoOutputFile():
    # Create headers/tags for each value in csv, and estalish them in Pandas dataframe
    DEFAULT_TAGS = ["step", "time", "temp", "press", "pe", "ke", "etotal","lx", "ly", "lz", "xy", "xz", "yz", "pxx", "pyy", "pzz", "pxy", "pxz", "pyz", "density"]
    def __init__(self, tags=None):
        self.tags = None
        self.data_df: pd.DataFrame = None
        if tags is None:
            self.tags = LammpsThermoOutputFile.DEFAULT_TAGS
        else:
            self.tags = tags
            
    def read(self, path):
        with open(path, 'r') as f:
            lines = f.readlines()
        
        data = []
        for line in lines:
            if not line.startswith("#"):
                args = [k.strip() for k in line.strip().split()]
                data.append(args)
        
        self.data_df = pd.DataFrame(data=data, columns=self.tags)
        for i in range(len(LammpsThermoOutputFile.DEFAULT_TAGS)):
            n = LammpsThermoOutputFile.DEFAULT_TAGS[i]
            
            if n in ['step', "time"]:
                # convert int types to int
                self.data_df[n] = self.data_df[n].astype(int)
            else:
                # convert float types to float
                self.data_df[n] = self.data_df[n].astype(float)


                
# Function to create line-graph of columns of the data
def line_graph(fig_num, x, y, title, label=''):
    fig_name = "Fig" + fig_num
    figure = plt.figure(fig_name)
    plt.plot(data[x], data[y], label=label)
    plt.title(title)

# Function to create scatter-graph of columns of the data
def scatt_graph(fig_num, x, y, title, size, color):
    fig_name = "Fig" + fig_num
    figure = plt.figure(fig_name)
    plt.scatter(data[x], data[y], s=size, c=color, marker=".")
    plt.title(title)
    
# Function to graph linear regression
def line_Reg(x, y):
    # math to find regression line
    m = (((mean(data[x])*mean(data[y])) - mean(data[x]*data[y])) /
         ((mean(data[x])*mean(data[y])) - mean(data[x]*data[y])))
    
    b = mean(data[y]) - m*mean(data[x])
    # Create line of best fit
    regression_line = [(m*i)+b for i in data[x]]
    
    # Plot line of best fit
    plt.plot(data[x], regression_line)


In [17]:
# Run classes and methods to read data and create Pandas dataframe
fileout_path = "2CEE_10/system.npt2CEE.300K.300K.out"
file = LammpsThermoOutputFile()
file.read(fileout_path)
data = file.data_df

DL = data['density']

# Print data
# print(data)

In [20]:
# Stats and analysis

DL = data['density']
avg_density = DL.mean()
print("The average density is:\n", avg_density)

stdv = DL.std()
print("\n\nThe standard deviation is:\n", stdv)

print("\n\nPercent deviation is: ", stdv/avg_density, "%")

The average density is:
 0.6683454410383425


The standard deviation is:
 0.382405843200141


Percent deviation is:  0.5721679534553789 %


In [21]:
# Calculations for determining mol ratio
def mol_ratio(dens1, molmass1, dens2, molmass2):
    mol1 = (1/molmass1) * (dens1) * 1000
    mol2 = (1/molmass2) * (dens2) * 1000
    perc1 = mol1/(mol1+mol2)
    perc2 = mol2/(mol1+mol2)
    
    mol1num = round(perc1)
    mol2num = round(perc2)
    
    return perc1, perc2

def mol_num(perc1, perc2, fact):
    molnum1 = round(perc1 * fact)
    molnum2 = round(perc2 * fact)
    
    return molnum1, molnum2


In [22]:
# mol ratio for 2CEE and LiPF6
perc1 = mol_ratio(avg_density, 124.14, 1/1000,1)
perc2 = mol_ratio(0.9955385211412047, 85.1, 1/1000,1)

print("Ratio for 2CEE and LIPF6:\n", perc1)
print("\nRatio for 3MPN and LIPF5:\n", perc2)

print("\n\n Mol 2CEE, Mol LiPF6:", mol_num(perc1[0], perc1[1], 2000/3))
print("\n\n Mol 3MPN, Mol LiPF6:", mol_num(perc2[0], perc2[1], 1000/3))


Ratio for 2CEE and LIPF6:
 (0.8433535891367955, 0.1566464108632045)

Ratio for 3MPN and LIPF5:
 (0.9212502623817903, 0.07874973761820969)


 Mol 2CEE, Mol LiPF6: (562, 104)


 Mol 3MPN, Mol LiPF6: (307, 26)


In [33]:
# # # plot

# # # 1st Figure: Time vs Energy: Individual graphs for KE, PE, and ME
# line_graph("1", "time", "ke", "Time vs Kinetic Energy", "Kinetic Energy")
# line_graph("1", "time", "pe", "Time vs Potential Energy", "Potential Energy")
# line_graph("1", "time", "etotal", "Time vs Total Energy", "Total Energy")
# # Label Axes
# plt.xlabel("Time (ps)")
# plt.ylabel("Energy (eV)")


# # # Create Legend
# plt.legend(loc="upper left")
# plt.savefig('Time.Energy.png')
# plt.close()


# # # 2nd Figure: Time vs Length of box
# line_graph("2", "time", "lx", "Time vs Lx")
# plt.xlabel("Time (ps)")
# plt.ylabel("Length (Å)")
# plt.savefig('Time.Lx.png')
# plt.close()

# # # 3rd Figure: Time vs Density
# line_graph("3", "time", "density", "Time vs Density")
# plt.xlabel("Time (ps)")
# plt.ylabel("Density (g/cm^3)")
# plt.savefig('Time.Density.png')
# plt.close()

# # # 4th Figure: Time vs Temperature
# line_graph("4", "time", "temp", "Time vs Temperature")
# plt.xlabel("Time (ps)")
# plt.ylabel("Temperature (K)")
# plt.savefig('Time.Temp.png')
# plt.close()